In [50]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import time
import json
import os

In [43]:
def DataAggregator(path):
    text_dict = {}
    json_dict = {}
    label_df = pd.DataFrame(columns=['changes', 'positions','file'])

    for file in os.listdir(path):
        if file.endswith('.txt'):
            with open(path+file, 'r', encoding='utf-8') as myfile:
                text_dict[file.replace('problem-', '').replace('.txt', '')] = myfile.read().replace('\n', '')
        else:
            with open(path+file, 'r', encoding='utf-8') as myfile:
                data = json.load(myfile)
                json_dict[file.replace('problem-', '').replace('.truth', '')] = data['changes']

    df = pd.DataFrame(list(text_dict.items()), columns=['file', 'text'])
    label_df = pd.DataFrame(list(json_dict.items()), columns = ['file', 'changes'])

    output_df = df.merge(label_df,on='file')
    output_df = output_df.drop('file', axis = 1)
    return output_df

In [44]:
#Convert the data sets to data frames
test_df = DataAggregator('C:/Users/RSC/PyProjects/Data-Science/Data/pan18-style-change-detection-test-dataset-2018-01-31/')
training_df = DataAggregator('C:/Users/RSC/PyProjects/Data-Science/Data/pan18-style-change-detection-training-dataset-2018-01-31/')
validation_df = DataAggregator('C:/Users/RSC/PyProjects/Data-Science/Data/pan18-style-change-detection-validation-dataset-2018-01-31/')


In [45]:
#Split the data into text and labels
training_labels = training_df['changes']
validation_labels = validation_df['changes']
test_labels = test_df['changes']

training_text = list(training_df['text'])
validation_text = list(validation_df['text'])
test_text = list(test_df['text'])

print("Training Text: " + str(len(training_text)) + ", Training Labels: " + str(len(training_labels)))
print("Test Text: " + str(len(test_text)) + ", Test Labels: " + str(len(test_labels)))
print("Validation Text: " + str(len(validation_text)) + ", Validation Labels: " + str(len(validation_labels)))

Training Text: 2980, Training Labels: 2980
Test Text: 1352, Test Labels: 1352
Validation Text: 1492, Validation Labels: 1492


In [48]:
#Builds a pipeline
NB_pipeline = Pipeline([('vect', CountVectorizer(strip_accents = 'ascii', ngram_range=(1,5), analyzer='word', stop_words='english')),
                         ('tfidf', TfidfTransformer()),
                         ('nb_clf', MultinomialNB())])

SVC_pipeline = Pipeline([('vect', CountVectorizer(strip_accents = 'ascii', ngram_range=(1,5), analyzer='word', stop_words='english')),
                        ('tfidf', TfidfTransformer()),
                        ('svc_clf', SVC())])


In [ ]:
#Find best parameters
parameters = {'ngram_range': [(1,1), (1,2), (1,3), (1,4), (1,5)],
              'tfidf': (True, False),
              ''}

nb_search = GridSearch(NB_pipeline, parameters, n_jobs=-1)
nb_search = nb_search.fit(training_text, training_labels)

svc_search = GridSearch(SVC_pipeline, parameters, n_jobs=-1)
svc_search = svc_search.fit(training_text, training_labels)

print("Best NB Score: " =

In [49]:
#Train the models and fit to the validation set
start = time.time()
NB_pipeline.fit(training_text, training_labels)
nb_prediction = NB_pipeline.predict(validation_text)
print("Naive Bayes Accuracy: " + str(accuracy_score(nb_prediction, validation_labels)) + " -- Run time " + str(time.time()-start))

start = time.time()
SVC_pipeline.fit(training_text, training_labels)
svc_prediction = SVC_pipeline.predict(validation_text)
print("SVM Accuracy: " + str(accuracy_score(svc_prediction, validation_labels)) + " -- Run time " + str(time.time()-start))

Naive Bayes Accuracy: 0.554959785523 -- Run time 25.458515405654907
SVM Accuracy: 0.658847184987 -- Run time 114.53243255615234
